# Medianas - como NO debería ser
Implementación de la generación del compuesto temporal de medianas, sobre el cubo V2, utilizando las consultas de bajo nivel provistas por el cubo (no paraleliza, ni utliza las funciones definidas en los componentes GridWorkflow, Analytics ni Execution)
## Parámetros

In [1]:
longs=(-75, -74)
lats=(5, 6)
normalized=True
time_range=("2000-01-01","2000-12-31")
bands=["blue","green","red","nir", "swir1","swir2"]
#bands=["blue"]
minValid=1;

## Consulta

In [2]:
%matplotlib inline
import datacube
import numpy as np
dc = datacube.Datacube(app='dc-prueba1')
nbar = dc.load(product='ls7_ledaps_utm', longitude=longs, latitude=lats, time=time_range)

In [3]:
## Opcional (para ver los resultados en el cuaderno de python)
nbar

<xarray.Dataset>
Dimensions:            (latitude: 3687, longitude: 3687, time: 7)
Coordinates:
  * time               (time) datetime64[ns] 2000-02-04T15:05:07 ...
  * latitude           (latitude) float64 6.0 6.0 5.999 5.999 5.999 5.999 ...
  * longitude          (longitude) float64 -75.0 -75.0 -75.0 -75.0 -75.0 ...
Data variables:
    blue               (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    green              (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    red                (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    nir                (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    swir1              (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    swir2              (time, latitude, longitude) int16 -9999 -9999 -9999 ...
    cf_mask            (time, latitude, longitude) int16 255 255 255 255 255 ...
    cf_mask_conf       (time, latitude, longitude) int16 255 255 255 255 255 ...
    adjacent_cloud_qa  (time

## Medianas:

In [4]:
nodata=-9999
medians={}
for band in bands:
    
    datos=np.where(nbar.data_vars[band]!=nodata,nbar.data_vars[band], np.nan)
    allNan=~np.isnan(datos)
    if normalized:
        m=np.nanmean(datos.reshape((datos.shape[0],-1)), axis=1)
        st=np.nanstd(datos.reshape((datos.shape[0],-1)), axis=1)
        datos=np.true_divide((datos-m[:,np.newaxis,np.newaxis]), st[:,np.newaxis,np.newaxis])*np.nanmean(st)+np.nanmean(m)
    medians[band]=np.nanmedian(datos,0)
    medians[band][np.sum(allNan,0)<minValid]=np.nan
del datos    

/home/developer/anaconda2/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:740: RuntimeWarning: All-NaN slice encountered
  warnings.warn("All-NaN slice encountered", RuntimeWarning)


In [5]:
# Opcional (para ver los resultados en el cuaderno de python)
medians

{'blue': array([[           nan,  6970.64450932,  6576.93792827, ...,
          3809.17350076,  3776.15992878,            nan],
        [           nan,  7688.13295046,  7596.92679269, ...,
          3891.7074307 ,  3792.66671477,            nan],
        [           nan,  7983.03286059,  8451.22447049, ...,
          3924.72100267,  3825.68028674,            nan],
        ..., 
        [           nan,            nan,            nan, ...,
          8775.00633058,  8749.16458588,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan]]),
 'green': array([[           nan,  6031.28917989,  5688.64572794, ...,
          3611.88644401,  3593.07562836,            nan],
        [           nan,  6942.33648906,  6839.863681  , ...,
          3709.70268538,  3650.44861609,            nan],
       

## Guardar los resultados a un archivo NetCDF

In [6]:
from datacube.storage import netcdf_writer
from datacube.model import Variable, CRS
nco=netcdf_writer.create_netcdf("salida2000.nc")
cords=('latitude', 'longitude','time')
for x in cords:
    if(x!="time"):
        netcdf_writer.create_coordinate(nco, x, nbar.coords[x].values, nbar.coords[x].units)
    else:
        netcdf_writer.create_coordinate(nco, x, np.array(["t1-t2"]), "period")
netcdf_writer.create_grid_mapping_variable(nco, nbar.crs)
for band in bands:
    medians[band][np.isnan(medians[band])]=nodata
    var= netcdf_writer.create_variable(nco, band, Variable(np.dtype(np.int32), None, ('time','latitude', 'longitude'), None) ,set_crs=True)
    var[:] = netcdf_writer.netcdfy_data(medians[band])
nco.close()